# 1. Datasets from original article

### Load and select columns

In [2]:
import pandas as pd
import math
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
df_train = pd.read_excel ('time_series_375_prerpocess_en.xlsx')

_id = 1 # filling ID gaps
for i in range(1, len(df_train)):
    if math.isnan(df_train.iloc[i, 0]):
        df_train.iloc[i, 0] = _id
    else:
        _id += 1
        
df_train = df_train[["PATIENT_ID", "RE_DATE", "outcome", "Lactate dehydrogenase", "High sensitivity C-reactive protein", "(%)lymphocyte"]]
df_train.columns = ["id", "test_date", "outcome", "lactate", "c_protein", "lymphocyte"]

grouped_train = df_train.groupby("id")

In [4]:
df_test = pd.read_excel ('time_series_test_110_preprocess_en.xlsx')

_id = 1 # filling ID gaps
for i in range(1, len(df_test)):
    if math.isnan(df_test.iloc[i, 0]):
        df_test.iloc[i, 0] = _id
    else:
        _id += 1
        
df_test = df_test[["PATIENT_ID", "RE_DATE", "outcome", "Lactate dehydrogenase", "High sensitivity C-reactive protein", "(%)lymphocyte"]]
df_test.columns = ["id", "test_date", "outcome", "lactate", "c_protein", "lymphocyte"]

grouped_test = df_test.groupby("id")

df_test

,id,test_date,outcome,lactate,c_protein,lymphocyte
0,1.0,2020-02-10 11:15:00,0,NaN,NaN,28.0
1,1.0,2020-02-10 12:07:00,0,183.0,5.1,NaN
2,1.0,2020-02-13 09:35:00,0,NaN,NaN,28.6
3,1.0,2020-02-13 10:06:00,0,194.0,2.7,NaN
4,2.0,2020-02-09 06:56:00,0,NaN,NaN,28.3
...,...,...,...,...,...,...
752,110.0,2020-02-05 12:36:00,0,562.0,78.1,NaN
753,110.0,2020-02-11 09:40:00,0,NaN,NaN,9.7
754,110.0,2020-02-11 10:04:00,0,438.0,NaN,NaN
755,110.0,2020-02-17 10:00:00,0,430.0,31.2,NaN


### Selecting suitable observations

In [5]:
df_new_train = pd.DataFrame()

for id, group in grouped_train:
        
    patients_doubled = group.loc[group['lactate'].isnull().values== False].head(2)
    lactate_double = group.loc[group['lactate'].isnull().values== False].head(2)
    protein_double = group.loc[group['c_protein'].isnull().values== False].head(2)
    lymphocyte_double = group.loc[group['lymphocyte'].isnull().values== False].head(2)

    if (len(protein_double.c_protein) == 2) and (len(lactate_double.lactate) == 2) and (len(lymphocyte_double.lymphocyte) == 2):
                
        
        patients_doubled["lymphocyte_change"] = (lymphocyte_double.lymphocyte.iloc[1] - lymphocyte_double.lymphocyte.iloc[0]) 
        patients_doubled["lactate_change"] = (lactate_double.lactate.iloc[1] - lactate_double.lactate.iloc[0]) 
        patients_doubled["c_protein_change"] = (protein_double.c_protein.iloc[1] - protein_double.c_protein.iloc[0])


    
        patients_doubled.lymphocyte = lymphocyte_double.lymphocyte.values        
        patients_doubled["lactate"] = lactate_double.lactate.values
        patients_doubled["c_protein"] = protein_double.c_protein.values


        
        
        df_new_train = df_new_train.append(patients_doubled.iloc[1]) 

In [6]:
df_new_test = pd.DataFrame()

for id, group in grouped_test:
        
    patients_doubled = group.loc[group['lactate'].isnull().values== False].head(2)
    lactate_double = group.loc[group['lactate'].isnull().values== False].head(2)
    protein_double = group.loc[group['c_protein'].isnull().values== False].head(2)
    lymphocyte_double = group.loc[group['lymphocyte'].isnull().values== False].head(2)

    if (len(protein_double.c_protein) == 2) and (len(lactate_double.lactate) == 2) and (len(lymphocyte_double.lymphocyte) == 2):
                        
        patients_doubled["lymphocyte_change"] = (lymphocyte_double.lymphocyte.iloc[1] - lymphocyte_double.lymphocyte.iloc[0])
        patients_doubled["lactate_change"] = (lactate_double.lactate.iloc[1] - lactate_double.lactate.iloc[0])
        patients_doubled["c_protein_change"] = (protein_double.c_protein.iloc[1] - protein_double.c_protein.iloc[0])


    
        patients_doubled.lymphocyte = lymphocyte_double.lymphocyte.values        
        patients_doubled["lactate"] = lactate_double.lactate.values
        patients_doubled["c_protein"] = protein_double.c_protein.values


        
        
        df_new_test = df_new_test.append(patients_doubled.iloc[1]) 


df_new_test

,c_protein,c_protein_change,id,lactate,lactate_change,lymphocyte,lymphocyte_change,outcome,test_date
3,2.7,-2.4,1.0,194.0,11.0,28.6,0.6,0.0,2020-02-13 10:06:00
7,0.7,-4.6,2.0,172.0,-44.0,30.5,2.2,0.0,2020-02-16 11:39:00
10,50.9,16.2,3.0,669.0,303.0,3.6,-8.4,1.0,2020-01-31 10:30:00
28,1.9,-2.0,4.0,198.0,17.0,30.5,12.4,0.0,2020-02-08 10:18:00
37,19.5,6.2,5.0,261.0,38.0,33.3,23.8,0.0,2020-01-30 13:04:00
...,...,...,...,...,...,...,...,...,...
684,49.0,-77.8,106.0,650.0,88.0,6.0,-11.2,1.0,2020-02-07 17:51:00
720,0.4,-6.8,107.0,152.0,-64.0,42.7,25.6,0.0,2020-02-14 10:02:00
724,2.3,-12.1,108.0,195.0,-68.0,30.4,0.2,0.0,2020-02-12 09:41:00
732,105.3,-18.5,109.0,501.0,-8.0,1.9,0.4,1.0,2020-02-07 17:54:00


In [7]:
df_new_test.to_csv("Dane_test.csv", index=False)
df_new_train.to_csv("Dane.csv", index=False)

# 2. Dataset from https://www.nature.com/articles/s42256-020-00252-4#Sec2

In [9]:
df_test_2 = pd.read_excel ("42256_2020_252_MOESM2_ESM.xlsx")
df_test_2 = df_test_2[["Death_D28", "crp_J1", "crp_J2", "crp_J3", "crp_J4", "crp_J5",
                       "ldh_j1", "ldh_j2", "ldh_j3", "ldh_j4", "ldh_j5",
                       "L_pourc_J1", "L_pourc_J2","L_pourc_J3","L_pourc_J4","L_pourc_J5"]]
df_test_2

,Death_D28,crp_J1,crp_J2,crp_J3,crp_J4,crp_J5,ldh_j1,ldh_j2,ldh_j3,ldh_j4,ldh_j5,L_pourc_J1,L_pourc_J2,L_pourc_J3,L_pourc_J4,L_pourc_J5
0,0,174.0,.,.,.,.,.,465.00,457.00,412.00,362.0,25.0000,.,.,.,.
1,1,126.0,.,.,.,.,652.0,900.00,.,.,.,.,13.3333,.,.,.
2,1,173.0,.,.,.,.,498.0,505.00,.,.,596.0,.,.,.,.,.
3,1,243.0,.,.,.,.,568.0,.,.,.,.,4.1667,.,.,.,.
4,1,.,.,451.0,.,.,455.0,450.00,535.00,.,497.0,10.0000,.,.,4.1176,.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,0,181.1,.,237.6,.,239.5,503.0,.,.,.,.,4.7619,.,6.1224,.,5.2632
174,0,293.4,.,.,124.7,.,466.0,552.00,599.00,.,.,1.9417,.,.,6.1856,6.2500
175,0,43.4,28.6,.,.,.,550.0,.,.,.,.,15.7895,13.2075,.,.,.
176,0,339.5,.,350.6,.,308.7,579.0,.,.,.,.,6.5041,.,8.6667,.,2.9630


In [10]:
data_check = pd.DataFrame(columns=df_test_2.columns)
count_crp = 0

for i in range (df_test_2.shape[1]):
    a = np.where(df_test_2.iloc[:,i] != ".", 1, 0)
    data_check.iloc[:,i] = a

data_check

data_check["crp_count"] = data_check.iloc[:, 1:6].sum(axis = 1)
data_check["ldh_count"] = data_check.iloc[:, 6:11].sum(axis = 1)
data_check["L_count"] = data_check.iloc[:, 11:16].sum(axis = 1)

valid_index =  data_check[(data_check.crp_count > 1) & (data_check.ldh_count > 1) & (data_check.L_count > 1) ].index
df_test_2_valid = df_test_2.iloc[valid_index,:]

In [11]:
df_test_2_valid

,Death_D28,crp_J1,crp_J2,crp_J3,crp_J4,crp_J5,ldh_j1,ldh_j2,ldh_j3,ldh_j4,ldh_j5,L_pourc_J1,L_pourc_J2,L_pourc_J3,L_pourc_J4,L_pourc_J5
7,1,175.0,.,201.0,.,.,.,709.00,738.00,776.00,897.0,12.8571,.,.,4.1176,.
8,1,.,199.0,297.0,308.0,208.0,.,488.00,445.00,381.00,342.0,.,11.0312,9.0000,3.1785,3.1051
9,0,127.0,.,132.0,145.0,64.0,327.0,318.00,294.00,276.00,239.0,9.7179,.,15.9631,16.9607,16.1905
10,0,74.0,88.0,75.0,75.0,.,.,275.00,272.00,257.00,.,.,12.8205,9.5571,9.1139,.
11,0,.,171.0,.,107.0,.,.,362.00,394.00,495.00,.,.,11.7220,.,28.1947,.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,1,300.4,.,.,315.6,.,669.0,.,.,.,3421.0,2.0408,5.7692,.,6.9231,7.3864
161,1,361.9,.,218.8,.,99.4,822.0,.,684.00,.,.,1.8182,1.3245,3.3113,.,4.0000
167,1,.,418.5,.,256.8,.,603.0,475.00,.,.,.,9.8592,.,.,3.3333,.
168,1,95.4,.,.,252.8,.,438.0,395.00,.,.,.,4.7619,4.5455,5.1282,5.6818,5.3333


In [14]:
df_new_2_test = pd.DataFrame(index=df_test_2_valid.index, columns = ["outcome", "c_protein", "c_protein_change", "lactate", "lactate_change", "lymphocyte", "lymphocyte_change"])

df_new_2_test["outcome"] = df_test_2_valid.Death_D28

# c_protein levels
for i in range(df_test_2_valid.shape[0]):
    c_1 = 0
    c_2 = 0
    for j in range (1, 6):
        
        c = df_test_2_valid.iloc[i,j]
        if c != "." and c_1 == 0:
            c_1 = c
            continue
        if c != "." and c_2 == 0:
            c_2 = c
    df_new_2_test.iloc[i, 1] = float(c_2)
    df_new_2_test.iloc[i, 2] = float(c_2) - float(c_1)

# ldh levels
for i in range(df_test_2_valid.shape[0]):
    ldh_1 = 0
    ldh_2 = 0
    for j in range (6, 11):
        
        ldh = df_test_2_valid.iloc[i,j]
        if ldh != "." and ldh_1 == 0:
            ldh_1 = ldh
            continue
        if ldh != "." and ldh_2 == 0:
            ldh_2 = ldh
            
    df_new_2_test.iloc[i, 3] = float(ldh_2)
    df_new_2_test.iloc[i, 4] = float(ldh_2) - float(ldh_1)
    
# lymphocyte levels
for i in range(df_test_2_valid.shape[0]):
    lymphocyte_1 = 0
    lymphocyte_2 = 0
    for j in range (6, 11):
        
        lymphocyte = df_test_2_valid.iloc[i,j]
        if lymphocyte != "." and lymphocyte_1 == 0:
            lymphocyte_1 = lymphocyte
            continue
        if lymphocyte != "." and lymphocyte_2 == 0:
            lymphocyte_2 = lymphocyte
            
    df_new_2_test.iloc[i, 5] = float(lymphocyte_2)
    df_new_2_test.iloc[i, 6] = float(lymphocyte_2) - float(lymphocyte_1)
df_new_2_test

,outcome,c_protein,c_protein_change,lactate,lactate_change,lymphocyte,lymphocyte_change
7,1,201,26,738,29,738,29
8,1,297,98,445,-43,445,-43
9,0,132,5,318,-9,318,-9
10,0,88,14,272,-3,272,-3
11,0,107,-64,394,32,394,32
...,...,...,...,...,...,...,...
156,1,315.6,15.2,3421,2752,3421,2752
161,1,218.8,-143.1,684,-138,684,-138
167,1,256.8,-161.7,475,-128,475,-128
168,1,252.8,157.4,395,-43,395,-43


In [15]:
df_new_2_test.to_csv("Dane_test2.csv", index=False)

# 3. Dataset from https://www.nature.com/articles/s42256-020-00254-2

In [5]:
'df_test_3 = pd.read_csv("Yan_reply_First_last_wtime.csv")'

'df_test_3 = pd.read_csv("Yan_reply_First_last_wtime.csv")'

In [19]:
df_test_3["lactate_change"] = df_test_3.Last_LDH - df_test_3.First_LDH
df_test_3["c_protein_change"] = df_test_3.Last_CRP - df_test_3.First_CRP
df_test_3["lymphocyte_change"] = df_test_3.Last_Lymph - df_test_3.First_Lymph
df_test_3['outcome'] = df_test_3.Expired_Outcome
df_test_3['lactate'] = df_test_3.Last_LDH
df_test_3['c_protein'] = df_test_3.Last_CRP
df_test_3['lymphocyte'] = df_test_3.Last_Lymph
df_test_3.to_csv("Dane_test3.csv", index=False)